In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
ques_df=pd.read_pickle("ques.pkl")
print(ques_df.shape)
feature_df=pd.read_pickle('final.pkl')
print(feature_df.shape)

(40000, 2)
(40000, 24)


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# merge texts

questions=list(ques_df['question1']) + (list(ques_df['question2']))
cv=CountVectorizer(max_features=3000)
q1_arr,q2_arr =np.vsplit(cv.fit_transform(questions).toarray(),2)

In [4]:
temp_df1=pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2=pd.DataFrame(q2_arr, index=ques_df.index)
temp_df=pd.concat([temp_df1,temp_df2],axis=1)
temp_df.shape

(40000, 6000)

In [5]:
final_df=pd.concat([feature_df,temp_df], axis=1)

print(final_df.shape)
final_df.head()

(40000, 6024)


,is_duplicate,q1_len,q2_len,q1_num_of_words,q2_num_of_words,word_common,word_total,words_share,cwc_min,cwc_max,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
8067,0,33,33,8,8,7,16,0.438,0.749981,0.749981,...,0,0,0,0,0,0,0,0,0,0
368101,0,52,50,11,9,6,20,0.300,0.399992,0.399992,...,0,0,0,0,0,0,0,0,0,0
70497,0,95,44,17,8,2,25,0.080,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
226567,1,43,51,7,9,6,16,0.375,0.999975,0.999975,...,0,0,0,0,0,0,0,0,0,0
73186,1,37,28,9,7,4,16,0.250,0.749981,0.599988,...,0,0,0,0,0,0,0,0,0,0


In [6]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
8067,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
368101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
226567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73186,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0].values,test_size=0.2,random_state=42)

In [8]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((32000, 6023), (8000, 6023), (32000,), (8000,))

In [15]:
# Random forest Model

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf=RandomForestClassifier(
    n_jobs=-1
)
rf.fit(X_train,y_train)
y_pred_rf=rf.predict(X_test)
accuracy_score(y_test,y_pred_rf)

0.79

In [16]:
# XGBoost Classifier

from xgboost import XGBClassifier

xgb=XGBClassifier(
        n_jobs=-1
)
xgb.fit(X_train,y_train)
y_pred_xgb=xgb.predict(X_test)
accuracy_score(y_test,y_pred_xgb)

0.79775

In [17]:
# LightGBM Classifier

from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(
        n_jobs=-1
)
lgbm.fit(X_train,y_train)
y_pred_lgbm=lgbm.predict(X_test)
accuracy_score(y_test,y_pred_lgbm)

[LightGBM] [Info] Number of positive: 11843, number of negative: 20157
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.252668 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12104
[LightGBM] [Info] Number of data points in the train set: 32000, number of used features: 3444
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.370094 -> initscore=-0.531815
[LightGBM] [Info] Start training from score -0.531815


/media/sourav/storage/ML_code/mlcd/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


0.80125

In [18]:
from sklearn.metrics import confusion_matrix

# for random forest

print("Confusion Matrix for Random Forest: \n",confusion_matrix(y_test,y_pred_rf))

# For XGBoost

print("Confusion Matrix for XGBoost Model: \n",confusion_matrix(y_test,y_pred_xgb))

# for LightGBM

print("Confusion Matrix for LightGBM: \n",confusion_matrix(y_test,y_pred_lgbm))

Confusion Matrix for Random Forest: 
 [[4331  736]
 [ 944 1989]]
Confusion Matrix for XGBoost Model: 
 [[4270  797]
 [ 821 2112]]
Confusion Matrix for LightGBM: 
 [[4251  816]
 [ 774 2159]]
